TODO :

definir un modèle probabiliste "conditionnel" hiérarchique

et le tester en premier avec info complete (tous les pixels)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from waveimage import WaveImage

In [ ]:
import math

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

In [ ]:
image = mnist.train.images[1].reshape(28,28)

image = np.append(np.zeros((2,28)), image, axis = 0)
image = np.append(image, np.zeros((2,28)), axis = 0)
image = np.append(np.zeros((32,2)), image, axis = 1)
image = np.append(image, np.zeros((32,2)), axis = 1)


In [ ]:
#image = image[:3,:]
w = WaveImage(image = image)


In [ ]:
plt.imshow(w.getImage())

In [ ]:
print w

In [ ]:
print w

In [ ]:
print w

In [ ]:
print w

In [ ]:
print w

In [ ]:
import pywt
pywt.wavedec2(image, 'haar')[1]

In [ ]:
min(int(math.log(28, 2)) + 1, 	int(math.log(3, 2)) + 1)

In [ ]:
28 / 2, 3 / 2

In [ ]:
image.shape

In [ ]:
2**5

In [ ]:
math.ceil(28. / 2**4) 

In [ ]:
math.ceil(32. / 2**5) 

In [ ]:
28. / 2**4

In [ ]:
math.ceil(1.75)